In [1]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import Conspecific_Tracking_ta_edit_117 as cs
from datetime import datetime
from scipy import interpolate 
from scipy import misc
from beating import rotation
from beating import tail_spline
from functions import *
from hmmlearn import hmm
from auto_filter_full import  auto_scoring_tracefilter_full, transform_data
from functools import partial
from find_features import Feature_extraction

Load Data

In [2]:
path = "h5files/h5 2/IM1_IM2_2.1.1_LDLC_resnet50_DLC_toptrackFeb27shuffle1_170000.h5"
f = pd.HDFStore(path,'r')
df = f.get('df_with_missing')
df.columns = df.columns.droplevel()

Filter df(Yuyang's method, with 2 more columns used)

In [4]:
new_features=Feature_extraction(starttime=0,endtime=216062)
filtered_df=new_features.filter_df(df,add_midpoint=True)

/Users/ryan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/Users/ryan/Desktop/Fish Project/Scripts/auto_filter_full.py:37: RuntimeWarning: invalid value encountered in greater
  head_dist_check=head_dist>25 #assume normal, use 3 \sigma rule
/Users/ryan/Desktop/Fish Project/Scripts/auto_filter_full.py:63: RuntimeWarning: invalid value encountered in arccos
  angle=np.arccos(cos)/np.pi*180;angle_check=np.logical_or(np.logical_and(np.invert(np.isnan(angle)),angle>75),safety_check)
/Users/ryan/Desktop/Fish Project/Scripts/auto_filter_full.py:63: RuntimeWarning: invalid value encountered in greater
  angle=np.arccos(cos)/np.pi*180;angle_check=np.logical_or(np.logical_and(np.invert(np.isnan(angle)),angle>75)

compute features, export it into 4 dataframes

In [5]:
new_features.fit(filtered_df,filter_feature=True,fill_na=True,estimate_na=True)
other_features,curvature,diff_curvature,tangent=new_features.export_df()

/Users/ryan/Desktop/Fish Project/Scripts/functions.py:262: RuntimeWarning: invalid value encountered in greater
  det=head_ori[:,1]>0
/Users/ryan/Desktop/Fish Project/Scripts/find_features.py:129: RuntimeWarning: invalid value encountered in greater
  ori[abs(ori-np.nanmean(ori,axis=0))>3*np.nanstd(ori,axis=0)]=np.nan
/Users/ryan/Desktop/Fish Project/Scripts/find_features.py:130: RuntimeWarning: invalid value encountered in greater
  turn_angle[abs(turn_angle-np.nanmean(turn_angle,axis=0))>3*np.nanstd(turn_angle,axis=0)]=np.nan


In [6]:
other_features

,operculum,orientation,movement_speed,turning_angle
0,58.359462,43.856672,80.344808,15.779898
1,58.359462,43.856672,80.344808,15.779898
2,58.359462,43.856672,80.344808,15.779898
3,58.359462,43.856672,80.344808,15.779898
4,51.898460,36.362364,80.344808,15.779898
...,...,...,...,...
216057,44.742523,149.140012,0.397665,0.414681
216058,45.147371,149.224544,4.224872,0.176317
216059,44.880240,148.445126,28.151774,1.346669
216060,44.653703,148.357810,2.932637,1.827402


In [7]:
curvature

,curvature_head,curvature_spine1,curvature_spine1.5,curvature_spine2,curvature_spine2.5,curvature_spine3,curvature_spine4,curvature_spine5,curvature_spine6,curvature_spine7
0,0.001847,-0.005822,-0.007907,0.038080,-0.000888,-0.033867,0.055461,0.044528,0.043925,-0.012055
1,0.001847,-0.005822,-0.007907,0.038080,-0.000888,-0.033867,0.055461,0.044528,0.043925,-0.012055
2,0.001847,-0.005822,-0.007907,0.038080,-0.000888,-0.033867,0.055461,0.044528,0.043925,-0.012055
3,0.001847,0.024866,-0.007907,0.038080,0.005377,-0.004509,-0.061825,0.173976,0.065296,-0.012055
4,0.036960,0.006138,-0.007907,0.038080,-0.045726,0.106103,-0.076688,0.231674,0.036176,-0.026544
...,...,...,...,...,...,...,...,...,...,...
216057,-0.010904,-0.001997,0.001090,-0.002366,0.006386,-0.023215,0.029016,-0.004201,-0.003859,-0.003514
216058,-0.011938,-0.001559,0.002069,-0.006721,0.005884,-0.016851,0.017728,0.004058,-0.007743,-0.018360
216059,-0.012753,-0.001495,0.002313,-0.007764,-0.002432,0.017547,-0.019738,0.023280,-0.006363,-0.026617
216060,-0.011783,-0.002107,0.001143,-0.002465,-0.003606,0.016937,-0.019677,0.023608,-0.006761,-0.027807


Try Yuqi's method

In [3]:
new_features=Feature_extraction(starttime=0,endtime=216062)
filtered_df=new_features.filter_df(df,add_midpoint=False)

/Users/ryan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:8767: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [4]:
new_features.fit(filtered_df,filter_feature=True,fill_na=True,estimate_na=True)

/Users/ryan/Desktop/Fish Project/Scripts/functions.py:262: RuntimeWarning: invalid value encountered in greater
  det=head_ori[:,1]>0
/Users/ryan/Desktop/Fish Project/Scripts/find_features.py:129: RuntimeWarning: invalid value encountered in greater
  ori[abs(ori-np.nanmean(ori,axis=0))>3*np.nanstd(ori,axis=0)]=np.nan
/Users/ryan/Desktop/Fish Project/Scripts/find_features.py:130: RuntimeWarning: invalid value encountered in greater
  turn_angle[abs(turn_angle-np.nanmean(turn_angle,axis=0))>3*np.nanstd(turn_angle,axis=0)]=np.nan


In [5]:
other_features,curvature,diff_curvature,tangent=new_features.export_df()

In [6]:
other_features

,operculum,orientation,movement_speed,turning_angle
0,58.359462,43.856672,93.480250,17.917419
1,58.359462,43.856672,93.480250,17.917419
2,93.405234,20.204055,93.480250,17.917419
3,46.739699,43.604159,237.966269,17.917419
4,51.898460,36.362364,131.088275,17.917419
...,...,...,...,...
216057,44.742523,149.140012,0.397665,0.414681
216058,45.147371,149.224544,4.224872,0.176317
216059,44.880240,148.445126,28.151774,1.346669
216060,44.653703,148.357810,2.932637,1.827402


In [7]:
diff_curvature

,diff_curvature_head,diff_curvature_spine1,diff_curvature_spine2,diff_curvature_spine3,diff_curvature_spine4,diff_curvature_spine5,diff_curvature_spine6,diff_curvature_spine7
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.001135
1,0.000000,-0.006468,0.000000,0.043268,-0.026532,0.074045,-0.016148,-0.001135
2,0.000000,-0.005591,0.000000,-0.021588,0.037724,0.074045,-0.004283,0.050675
3,-0.023353,-0.009199,0.031446,-0.010657,-0.088722,0.074045,-0.004283,0.050675
4,0.030675,0.012323,-0.032995,0.013273,0.006892,0.002119,-0.004283,0.050675
...,...,...,...,...,...,...,...,...
216057,0.001409,0.000119,-0.000806,0.000718,-0.000732,0.000206,0.001495,0.000935
216058,0.003981,-0.000383,-0.003410,0.005079,-0.009226,0.001282,0.019959,0.000935
216059,0.009221,-0.001290,-0.008491,0.026215,-0.032117,0.003045,0.044767,0.000935
216060,-0.003499,0.000195,0.002673,-0.001243,0.000278,-0.000009,0.000203,0.000935
